In [127]:
from os.path import dirname, join
from os import listdir, getcwd

import pickle
import pandas as pd 
import sys
import pickle as pkl

PATH_REPO = getcwd()

PATH_DATA = join(PATH_REPO, 'data', '20230721_transcriptions')
PATH_LINES = join(PATH_DATA, 'lines')
PATH_XML = join(PATH_DATA, 'XML')
PATH_OCRized = join(PATH_DATA, 'OCRized')
PATH_PAGES = join(PATH_DATA, 'pages')
PATH_UTILS = join(PATH_REPO, 'utils')

sys.path.append(PATH_UTILS)

from parsing_lines import *
from xml.dom import minidom

In [128]:
def get_marges_and_corps(PATH_OCRized, th=25):

    texts = [x for x in listdir(PATH_OCRized) if '.txt' in x]
    texts.sort()
    lengths = [len([x[:-1] for x in open(join(PATH_OCRized, text), 'r').readlines()]) for text in texts]
    
    marges = [x[:-1] for text, length in zip(texts, lengths) if length <=th
         for x in open(join(PATH_OCRized, text), 'r').readlines()]
    corps = [x[:-1] for text, length in zip(texts, lengths) if length >th
             for x in open(join(PATH_OCRized, text), 'r').readlines()]
    
    photo_marges = [text.split('_column')[0] for text, length in zip(texts, lengths) if length <=th
     for x in open(join(PATH_OCRized, text), 'r').readlines()]
    
    photo_corps = [text.split('_column')[0] for text, length in zip(texts, lengths) if length >th
         for x in open(join(PATH_OCRized, text), 'r').readlines()]
    
    return marges, corps, photo_marges, photo_corps

In [129]:
for commune in communes[-1:]:
    years = [x for x in listdir(join(PATH_REPO, 'data', '20230721_transcriptions', commune)) if '.' not in x]
    for year in years[-1:]:
        PATH_DATA = join(PATH_REPO, 'data', '20230721_transcriptions', commune, year)
        PATH_LINES = join(PATH_DATA, 'lines')
        PATH_XML = join(PATH_DATA, 'XML')
        PATH_OCRized = join(PATH_DATA, 'OCRized')
        PATH_PAGES = join(PATH_DATA, 'pages')

        marges, corps, photo_marges, photo_corps = get_marges_and_corps(PATH_OCRized, th=30)

In [130]:
from Levenshtein import ratio
from collections import Counter

def duplicated_from_levenshtein(marge):
    """
    Objective: with the marge content find out words that are most likely related to the owrd list
    
    Inputs:
        - marge, str: the marge text
    Outputs:
        - duplicated, dict: the dictionary of words that should probably be one of the word list
    """
    
    words_list = ['naissance', 'reconnaissance', 'mariage',
                    'décès','présentation']

    words = [x.lower() for marge in marges for x in marge.split()]
    words_count = Counter(words)

    words_th = {'naissance':0.8, 
                'reconnaissance':0.8, 
                'mariage':0.85, 
                'décès': 0.7, 
                'présentation':0.85}

    duplicated = {word: [] for word in words_list}

    duplicated['décès'].append('veces')
    duplicated['décès'].append('décet')
    duplicated['décès'].append('decel')
    duplicated['décès'].append('Décéd')
    duplicated['décès'].append('Décil')
    duplicated['décès'].append('Dècès')
    duplicated['naissance'].append('nousSANCE')
    duplicated['naissance'].append('noussance')
    duplicated['naissance'].append('naissane')
    duplicated['reconnaissance'].append('Lecon NaISSANCE')
    duplicated['reconnaissance'].append('reconaissance')
    duplicated['reconnaissance'].append('Remaissance')

    for word in words_list:
        for c_word in np.unique(words):
            r = ratio(word, c_word)
            if r > words_th.get(word) and r < 1:
                duplicated[word].append(c_word)
                
    return duplicated

import re

def modify_with_levenshtein(marges, duplicated):
    """
    Objective: replace the words in marge that are most likely related to words into duplicated dict
    
    Inputs:
        - marges, list: the list of raw texts
        - duplicated, dict: dictionnary with fixed vocabulary and in values the words it could have written as
    Outputs:
        - marges, list: the list of cleaned texts
    """
    for word, duplicateds in duplicated.items():
        for _duplicated in duplicateds:
            _duplicated = _duplicated.replace('(', '\(').replace(')', '\)')
            for i, marge in enumerate(marges):
                marge = re.sub(r'(?:{}) '.format(_duplicated), 
                               word.capitalize(), marge, flags=re.I)
                marges[i] = marge
                
    return marges

def split_with_acte(marges):
    """
    Objective: split all marges to get for each element one acte only
    
    Inputs:
        - marges, list: the list of texts
    Outputs:
        - actes, list: list of texts where one item is one acte
        - regex, str: the regex to improve consolidation afterwords
    """
    actes_words = ['D[ée]c[èe]s', 'Naissance\s*(?:et|)\s*reco(?:|n)naissance',
                   'Recon(?:n|)aissance','Mariage', 'Pr[ée]sentation', 'Naissance']

    regex ='(?:N°\s*(?:{})|(?:N°|N|)\s*(?:\d+|L|S|)\s*(?:{}))'.format(
            '|'.join(actes_words[:4]),
                 '|'.join(actes_words))

    actes = re.split(r'({})'.format(regex), ' '.join(marges).replace('n°', 'N°'), flags=re.I)

    return actes, regex

def get_photos_by_actes(actes, marges, photo_marges):
    """
    Objective: get the related photos to each actes
    
    Inputs:
        - actes, list: list of texts where one item is one acte
        - marges, list: the list of texts
        - photos_marges, list: the list of photos associated
    Outputs:
        - photo_marges_actes, list: final list of the photos
    """

    i = 0
    j = 0
    k = 1
    photo_marges_actes = []
    merged_texts = []
    initial_text = ''
    
    while j < len(actes) and i < len(marges):
        text = actes[j]
        merged_text = (initial_text + ' ' + marges[i].replace('n°', 'N°').strip()).strip()

        while len(merged_text.strip()) < len(text.strip()) and i+k < len(marges) - 1:
            merged_text += ' ' + marges[i+k].replace('n°', 'N°').strip()
            k += 1

        initial_text = merged_text.strip()[len(text.strip()):]
        merged_text = merged_text.strip()[:len(text.strip())]

        photo_marges_actes.append(np.unique(photo_marges[i:i+k])[0])
        merged_texts.append(merged_text)

        i += k
        k = 1
        j += 1
    
    return photo_marges_actes


def consolidate_actes(actes, photo_marges_actes, regex):
    """
    Objective: from the actes and regex get the final actes, with one item is one acte
    
    Inputs:
        - actes, list: list of texts
        - regex, str: the regexes to help divide texts
    Outputs:
        - final_actes, list: the list of final actes
    """
    final_actes, final_photos = [], []

    final_acte = ''
    photo = None
    for acte, photo in zip(actes, photo_marges_actes):
        matches = re.findall(regex, acte, flags=re.I)
        if len(matches) > 0:
            if len(final_acte.strip()) > 0:
                final_actes.append(final_acte)
                final_photos.append(photo)
            final_acte = acte.strip()
        else:
            final_acte += ' ' + acte.strip()
            
    final_actes.append(final_acte.strip())
    final_photos.append(photo)
    
    return final_actes, final_photos

def get_sexe(acte, nom, type_acte):
    """
    Objective: get the sex of the acte depending on the name and type acte
    
    Inputs:
        - acte, str: the act text
        - nom, str: the name of the person in the act
        - type_act, str: the type of act
    Outputs:
        - sexe, str: 'M', 'F' or None
    """
    sexe = ''
    if nom != '':
        s = acte.split(nom)[0]
        if any(y in s.lower() for y in ['sr', 'sieur']):
            sexe = 'M'
        if any(y in acte for y in ['delle', 'dlle', 'demoiselle']):
            sexe = 'F'
        
    if any(y in acte for y in ['fils d', 'fils lé', 'fils n', 'naturel ']):
        sexe = 'M'
    if any(y in acte for y in ['fille d', 'fille lé', 'fille n', 'naturelle']):
        sexe = 'F'
        
    if type_acte == 'Présentation' and 'sexe' in acte.lower():
        if 'sexe masculin' in acte.lower():
            sexe = 'M'
        if 'sexe féminin' in acte.lower():
            sexe = 'F'
    return sexe

def get_nom(acte):
    """
    Objective: rfom the acte and thype of acte get the name
    
    Inputs:
        - acte, str: the act text
        - type_acte, str: the type of act
    Outputs:
        - nom, str: the name of the person mentioned in the act
    """
    acte = acte.strip()
    acte = re.sub(r'^\d+', '', acte)
    nom = re.sub(r'(?:Présentation|Décès|Reco(?:n|)naissance|Mariage|Naissance)', '', acte, flags=re.I).strip()
    nom = re.sub(r'^d[eu] ', '', nom, flags=re.I).strip()
    words_to_delete = ['enfant', 'l[ée]gitime', 'reconnu']
    nom = re.sub('(?:{})'.format('|'.join(words_to_delete)), '', nom).replace('  ', ' ').strip()
    nom = re.sub("(?:l'|)archiviste(.*?)$", '', nom).strip()
    nom = re.sub("\w+ mot(s)* ray[ée](s)* nul(s)*(.*?)$", '', nom).strip()
    nom = re.sub("(?:reconnu|D[eé]clar[eé]|ce renvoi|l[ée]gitime|lequel|sur|à|DÉCÉdÉ|MARIé) (.*?)$", '', 
                 nom, flags=re.I).strip()
    nom = re.sub("le (?:un|premier|deux|trois|quatre|cinq|six|sept|huit|neuf|dix|onze|douze|treize|quatorze|seize|vingt|trente)(.*?)$",
                 "", nom, flags=re.I).strip()
    
    return nom

def consolidate_df(final_actes):
    """
    Objective: consolidate from all information the dataset of marges
    
    Inputs:
        - final_actes, list: list of one item is an act
    Outputs:
        - df, pd.DataFrame: the dataset as dataframe
    """
    df = pd.DataFrame(columns=['acte', 'type_acte', 'numéro', 'nom', 'sexe'])

    replaces = [('N°', ''), ('Entre', 'entre'), ('(', ''), (')', ''), ('Enfant', 'enfant'), ('L\'', 'l\''),
               ('Sexe', 'sexe'), ('Mas', 'mas'), ('Fém', 'fém'), ('Légitime', 'légitime'), ('Nous', 'nous'),
                ('Sieur', 'sieur'), ('Dlle', 'dlle'), ('Sr', 'sr'), ('Témo', 'témo'), ('Témi', 'témi'),
                ('Par ', 'par '), ('Marié', 'marié')]

    for k, acte in enumerate(final_actes):
        for rep in replaces:
            acte = acte.replace(rep[0], rep[1])
        acte = re.sub('reconaissance', 'Reconnaissance', acte, flags=re.I)
        type_acte = re.findall(r'Pr[ée]sentation|D[ée]c[èe]s|Naissance\s*(?:et|)\s*reco(?:|n)naissance|Reconnaissance|Mariage|Naissance', acte, flags=re.I)
        type_acte = type_acte[0].lower() if len(type_acte) > 0 else 'none'
        if re.search('d[ée]c[èe]s', type_acte):
            type_acte = 'décès'
        if re.search('pr[ée]sentation', type_acte):
            type_acte = 'présentation'
        if re.search('naissance\s*(?:et|)\s*reconnaissance', type_acte):
            type_acte = 'naissance'

        num = re.search(r'^\d+', acte.strip())
        num = num.group() if num else None

        nom = get_nom(acte.strip())

        sexe = get_sexe(acte, nom, type_acte)

        df.loc[k, ['acte', 'type_acte', 'numéro', 'nom', 'sexe']] = acte, type_acte, num, nom, sexe
    
    return df

In [131]:
communes = [x for x in listdir(join(PATH_REPO, 'data', '20230721_transcriptions'))
            if x[0] != '.' and 'xlsx' not in x and 'csv' not in x and '.gz' not in x]
communes

['Vieux Habitants',
 '8. Capesterre de MG',
 '23. Saint Esprit',
 '3. Basse Pointe',
 'Saint Louis',
 '1. Abymes',
 '14. Lorrain',
 "16. Morne à l'Eau",
 '21. Rivière Salée',
 '19. Prêcheur',
 '20. Rivière Pilote',
 '31. Trois Ilets',
 'Sainte Rose',
 'Sainte Anne',
 '14. Grand Bourg de MG',
 '1. Ajoupa Bouillon',
 'Trois Rivière',
 '17. Moule',
 '27. Sainte Luce',
 '20. Pointe à Pitre',
 '4. Carbet',
 '12. Gourbeyre',
 '24. Saint Joseph',
 'Pointe à pitre',
 '15. Macouba',
 '22. Robert',
 '16. Marigot']

In [132]:
df = pd.DataFrame()

for commune in communes:
    years = [x for x in listdir(join(PATH_REPO, 'data', '20230721_transcriptions', commune)) if '.' not in x]
    for year in years:
        PATH_DATA = join(PATH_REPO, 'data', '20230721_transcriptions', commune, year)
        PATH_LINES = join(PATH_DATA, 'lines')
        PATH_XML = join(PATH_DATA, 'XML')
        PATH_OCRized = join(PATH_DATA, 'OCRized')
        PATH_PAGES = join(PATH_DATA, 'pages')

        marges, corps, photo_marges, photo_corps = get_marges_and_corps(PATH_OCRized, th=30)
        duplicated = duplicated_from_levenshtein(marges)
        marges = modify_with_levenshtein(marges, duplicated)
        actes, regex = split_with_acte(marges)
        photo_marges_actes = get_photos_by_actes(actes, marges, photo_marges)
        final_actes, final_photos = consolidate_actes(actes, photo_marges_actes, regex)
        _df = consolidate_df(final_actes)
        _df.loc[:, 'picture'] = final_photos
        _df.loc[:, 'commune'] = commune
        _df.loc[:, 'year'] = year
        df = pd.concat((df, _df)).copy()

df.reset_index(drop=True, inplace=True)

In [133]:
len(df)

66943

In [134]:
df.groupby(['type_acte'], group_keys=True).agg({'acte': 'count'})

,acte
type_acte,
décès,23771
mariage,6580
naissance,31692
none,177
présentation,2045
reconnaissance,2678


In [135]:
len(df)

66943

In [136]:
df.groupby(['type_acte'], group_keys=True).agg({'acte': 'count'})

,acte
type_acte,
décès,23771
mariage,6580
naissance,31692
none,177
présentation,2045
reconnaissance,2678


In [137]:
df.loc[:, 'nom'] = df.loc[:, 'acte'].apply(lambda x: get_nom(x))

In [138]:
df.loc[(df.loc[:, 'numéro'] == '179') &
       (df.loc[:, 'commune'] == '3. Basse Pointe'), :]

,acte,type_acte,numéro,nom,sexe,picture,commune,year
8327,179 Décès du sieur Saint Lucien Nº,décès,179,sieur Saint Lucien Nº,,FRANOM21_1DPPC1255_154,3. Basse Pointe,1852
8588,179 Naissance de sieur Prêcheur François Narier,naissance,179,sieur Prêcheur François Narier,,FRANOM21_1DPPC1274_224,3. Basse Pointe,1896
8812,179 Naissance la sieur Calin Janville,naissance,179,la sieur Calin Janville,,FRANOM21_1DPPC1255_237,3. Basse Pointe,1853
8997,179 Décès de d'nicodil avrilleine,décès,179,d'nicodil avrilleine,,DAFCAOM04_DPPCEC_850477_0055,3. Basse Pointe,1854
9427,179 Naissance de la Dl Alton Domi- nique Marie...,naissance,179,la Dl Alton Domi- nique Marie 15 B 25 &5 F. S ...,,FRANOM21_1DPPC1274_060,3. Basse Pointe,1894
9708,179 Décès d'un enfant mort-né 180 Décédé lieu ...,décès,179,d'un mort-né 180,,FRANOM21_1DPPC1273_232,3. Basse Pointe,1893
9949,179 Naissance du sieur Guston Erem Bert Camille,naissance,179,sieur Guston Erem Bert Camille,,FRANOM21_1DPPC1273_141,3. Basse Pointe,1892
10217,179 Naissance et reconnait nier Jean Théodore ...,naissance,179,et reconnait nier Jean Théodore,,FRANOM21_1DPPC1274_139,3. Basse Pointe,1895


In [139]:
#~~~~~ as a split ?
df.loc[14547, 'acte']

"n NaISSANCE Joséphine Landre. Cultyloie et Sandre Lazarre Agénon par leur mère la demoiselle Lapalme Marie Noël Le 14 novembre 1892  et Treize mots rayés nuls l'Jean Ahouz Ile sart  Zueol"

In [140]:
df.loc[55873, 'acte']

'164 Décès de sieur Vallianne Gaësan Paul'

In [141]:
acte = ''
type_acte = re.findall(r'Pr[ée]sentation|D[ée]c[èe]s|Naissance\s*(?:et|)\s*reco(?:|n)naissance|Reconnaissance|Mariage|Naissance', acte, flags=re.I)

In [142]:
i = np.random.randint(0, len(df) - 50)
df.loc[i:i+50, :]

,acte,type_acte,numéro,nom,sexe,picture,commune,year
14506,"n Naissance Est comparu dont la cécontre, dont...",naissance,None,"n Est comparu dont la cécontre, dont mention f...",,FRANOM21_1DPPC0039_173,1. Abymes,1892
14507,142 NAISSANCE Marie Antonine Justine fille nat...,naissance,142,Marie Antonine Justine fille naturelle Le 7 Se...,F,FRANOM21_1DPPC0039_173,1. Abymes,1892
14508,143 NAISSANCE Rancel Adrien fils légitime Le 9...,naissance,143,Rancel Adrien fils Le 9 Septembre 1892,M,FRANOM21_1DPPC0039_173,1. Abymes,1892
14509,144 NAISSANCE d° Lamaille Victor Joseph fils r...,naissance,144,d° Lamaille Victor Joseph fils Le 17 septembre...,,FRANOM21_1DPPC0039_173,1. Abymes,1892
14510,145 NAISSANCE Janissois Lambert Certullien fil...,naissance,145,Janissois Lambert Certullien fils Le 18 septem...,,FRANOM21_1DPPC0039_174,1. Abymes,1892
14511,146 NAISSANCE Pierre Henri fils naturel Le 19 ...,naissance,146,Pierre Henri fils naturel Le 19 Septembre 1852,M,FRANOM21_1DPPC0039_174,1. Abymes,1892
14512,147 NAISSANCE Delrain Adélaïde Florence fille ...,naissance,147,Delrain Adélaïde Florence fille e Le 24 Septem...,F,FRANOM21_1DPPC0039_174,1. Abymes,1892
14513,148 NAISSANCE Beoffroy Henry Edmand fils légi...,naissance,148,Beoffroy Henry Edmand fils Le 29 septembre 1882,M,FRANOM21_1DPPC0039_174,1. Abymes,1892
14514,141 NAISSANCE 16 Haugie Prembert Bernadin fils...,naissance,141,16 Haugie Prembert Bernadin fils Le 30 Septemb...,M,FRANOM21_1DPPC0039_175,1. Abymes,1892
14515,150 NAISSANCE Bourgarelle Saint-Louis Zéphiri...,naissance,150,Bourgarelle Saint-Louis Zéphirin fils Le 4 Oct...,,FRANOM21_1DPPC0039_175,1. Abymes,1892


In [100]:
df.to_csv(join(PATH_REPO, 'data', '20230817_transcriptions', 'marges.csv'), index=False)

# Corps du texte

## Split the text in actes

Needs to improve after the next training.

In [103]:
def split_acte_in_corps(marges):
    """
    Objective. split the texts into actes
    """
    actes_words = ['Décès', 'Naissance\s*(?:et|)\s*reco(?:n|)naissance',
                   'Recon(?:n|)aissance','Mariage', 'Présentation', 'Naissance']

    regex = '(?:N°\s*(?:{})|(?:N°|N|)\s*(?:\d+\s*|L|S|)\s*(?:{}))'.format('|'.join(actes_words[:3]),
                 '|'.join(actes_words))

    actes = re.split(r'({})'.format(regex), ' '.join(marges).replace('n°', 'N°'), flags=re.I)

    return actes, regex

In [104]:
def duplicated_word_corps_from_levenshtein(corps):
    """
    Objective: get a dictionnary of words that should be replaced by the most common in our context (words_list)
    
    Inputs:
        - corps, list: list of texts
    Outputs:
        - ducplicated, dict: the dictionnary of words to replace misspelled ones
    """
    words_list = ["aujourd'hui", 'par-devant', 'naissance', 'décès', 'mariage']

    words = [x.lower() for marge in corps for x in marge.split()]
    words_count = Counter(words)

    words_th = {
                "aujourd'hui":0.9,
                "par-devant":0.8,
                'naissance':0.8, 
                'mariage':0.9, 
                'décès': 0.8
               }

    duplicated = {word: [] for word in words_list}
    
    for word in words_list:
        for c_word in np.unique(words):
            r = ratio(word, c_word)
            if r >= words_th.get(word) and r < 1:
                duplicated[word].append(c_word)
                
    return duplicated

def modify_with_levenshtein(marges, duplicated):
    """
    Objective: replace words in the text (marges) that are most likely present in the duplicated dict
    
    Inputs:
        - marges, list: list of texts
        - duplicated, dict: the words to focus on that may be a bit changed
    Outputs.
        - marges, list: list of cleaned texts
    """
    for word, duplicateds in duplicated.items():
        for _duplicated in duplicateds:
            _duplicated = _duplicated.replace('(', '\(').replace(')', '\)')
            for i, marge in enumerate(marges):
                marge = re.sub(r'\b(?:{}|{}|{})\b'.format(_duplicated, _duplicated.capitalize(), _duplicated.upper()), 
                               word.capitalize(), marge)
                marges[i] = marge
                
    return marges

def split_corps_in_actes(corps):
    """
    Objective: from the corps, try to have one item one acte
    
    Inputs:
        - corps, list: list of texts
    Outputs:
        - actes, list. the list of actes one items is an act
    """
    actes = []

    acte = ''
    for text in corps:
        if re.search("^(?:aujourd'hui|l'an )", text.lower()):
            if len(acte) < 500 and len(actes) > 0:
                actes[-1] += ' ' + acte
            else:
                actes.append(acte)
            acte = text
        else:
            acte += ' ' + text
            
    return actes

In [105]:
df = pd.DataFrame()

for commune in communes:
    years = [x for x in listdir(join(PATH_REPO, 'data', '20230721_transcriptions', commune)) if '.' not in x]
    for year in years:
        PATH_DATA = join(PATH_REPO, 'data', '20230721_transcriptions', commune, year)
        PATH_LINES = join(PATH_DATA, 'lines')
        PATH_XML = join(PATH_DATA, 'XML')
        PATH_OCRized = join(PATH_DATA, 'OCRized')
        PATH_PAGES = join(PATH_DATA, 'pages')

        marges, corps, photo_marges, photo_corps = get_marges_and_corps(PATH_OCRized, th=30)
        duplicated = duplicated_word_corps_from_levenshtein(corps)
        corps = modify_with_levenshtein(corps, duplicated)
        actes = split_corps_in_actes(corps)
        photo_corps_actes = get_photos_by_actes(actes, corps, photo_corps)
        if len(actes) < 1:
            continue
        _df = pd.DataFrame(actes, columns=['acte'])
        _df.loc[:, 'photo'] = photo_corps_actes
        _df.loc[:, 'commune'] = commune
        _df.loc[:, 'year'] = year
        df = pd.concat((df, _df)).copy()

df.reset_index(drop=True, inplace=True)

In [106]:
df

,acte,photo,commune,year
0,Premier juillet,FRANOM49_1DPPC0954_225,Vieux Habitants,1855
1,"Aujourd'hui, Samedi Six Janvier mil huit cent ...",FRANOM49_1DPPC0954_225,Vieux Habitants,1855
2,"Aujourd'hui, Lundi huit Janvier mil huit cent ...",FRANOM49_1DPPC0954_225,Vieux Habitants,1855
3,"Aujourd'hui, Lundi huit Janvier mil huit cent ...",FRANOM49_1DPPC0954_226,Vieux Habitants,1855
4,"Aujourd'hui, Samedi treize Janvier mil huit ce...",FRANOM49_1DPPC0954_226,Vieux Habitants,1855
...,...,...,...,...
76873,"L'an mil huit cent quatre-vingt-quinze, le hui...",FRANOM21_1DPPC1705_078,16. Marigot,1895
76874,"L'an mil huit cent quatre vingt-quinze, le neu...",FRANOM21_1DPPC1705_078,16. Marigot,1895
76875,"L'an mil huit cent quatre vingt-quinze, le onz...",FRANOM21_1DPPC1705_079,16. Marigot,1895
76876,"L'an mil huit cent quatre vingt quinze, le sei...",FRANOM21_1DPPC1705_079,16. Marigot,1895


In [110]:
for text in df.sample(10).acte.values:
    print(text)
    print('----')

L'an mil huit cent cinquante quatre et le premier du mois d'a¬ ind à huit heures du matin. Par-devant nous Frédéric sise en cide Latama le adjoint au Maire de Son d° d° Frêcheur arrondissement de Saint Pierre île Martinique aux présent les fonctions d'officier de l'Etat civil par délégation du Marin en date du premier Décembre dernier a comparu la demoiselle Collus Rose agée du dix neuf ans, fille naturelle de la demoiselle Collius Euphrasie âgée de quarante trois ans, toute deux cultivateurs domiciliés dans cette Commune. Lesquels nom de claré qu'il est né d'elle dans sa case Soir en Bourg de cette Commune, le Seize mars dernier à six heures du matin, un enfant du sexe masculin quelle nous présente et auquel elle déclare vouloir donner le prénom de Parul. Les dites déclaré que et présentation faites en présence des Sieurs Livorg Joseph, âgé de trente quatre ans et Gernet Romain agé de vingt trois ans, tous deux cultivateurs domiciliés dans cette Commune, voisins de la déclarante, témo

## Preprocess acte

In [111]:
df.loc[:, 'acte_pp'] = df.loc[:, 'acte'].apply(lambda x: x.replace('- -', '').replace(' -', '').replace('- ', ''))
df.loc[:, 'acte_pp'] = df.loc[:, 'acte_pp'].apply(lambda x: re.sub('[sS]ieur(?:s|)(?: |-)[hH]abitan(?:t|)(?:s|)', 
                                                                   'Vieux Habitants', x))
df.loc[:, 'acte_pp'] = df.loc[:, 'acte_pp'].apply(lambda x: re.sub('Vieux-habitation', 
                                                                   'Vieux Habitants', x))

## Extract key info:

- Date du registre
- Type évènement (mariage, décès, naissance, reconnaissance)
- Lieu évènement si présent
- Date évènement


- Sujet(s): qui est le sujet de l'acte
    * Nom
    * Age
    * Profession
    * Lieu de naissance
    * Résidence
    * Genre


- Père et/ou Mère:
    * nom
    * age
    * profession
    * lieu de naissance
    * en vie ?
    * résidence

### Info registre

**Date**

In [112]:
jours = ['un', 'deux', 'trois', 'quatre', 'cinq', 'six', 'sept',
         'huit', 'neuf', 'dix', 'onze', 'douze', 'treize', 
         'quatorze', 'quinze', 'seize', 'dix-sept', 'dix-huit', 
         'dix-neuf', 'vingt', 'vingt et un', 'vingt-deux', 
         'vingt-trois', 'vingt-quatre', 'vingt-cinq', 'vingt-six', 'vingt-sept', 
         'vingt-huit', 'vingt-neuf', 'trente', 'trente et un']
jours = [x.replace('-', '(?:-| )') for x in jours]
jours_ieme = [re.sub(r'e$', 'i[èe]me', x.replace('-', '(?:-| )')) for x in jours]
jours_ieme = [x.replace('f', 'v') + 'i[èe]me' if 'i[èe]me' not in x else x for x in jours_ieme]
jours_ieme[0] = 'premier'
annees = ['mil huit cent cinquante et un', 'mil huit cent cinquante deux', 
         'mil huit cent cinquante trois', 
         'mil huit cent cinquante quatre', 'mil huit cent cinquante cinq']
annees = [x.replace(' ', '(?:-|\s*|\s*d°\s*)') for x in annees]
mois = ['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 
        'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre']

In [113]:
i = np.random.randint(len(actes))

def get_date_registre(acte):
    """
    Objective get the date of the acte
    """
    date_texte = re.split(r'((?:[Pp]ar-devant|(?:est|sont) comparu))', acte)[0]
    date_texte = re.sub(r'(?:{})'.format('|'.join(annees)), 'ANNEE', date_texte.lower())
    
    m = re.search(r'(?:{})\b'.format('|'.join(mois)), date_texte.lower())

    if m:
        j = re.search(r'(?:{})'.format('|'.join(jours_ieme[::-1] + jours[::-1])), 
                         date_texte.lower()[:m.start()])
        m = m.group()
        if j:
            j = j.group()
        else:
            j = None
    else:
        m = None
        j = re.search(r'(?:{})'.format('|'.join(jours_ieme[::-1] + jours[::-1])), 
                         date_texte.lower())
        if j:
            j = j.group()
        else:
            j = None
        
        
    if not m:
        splits = re.split(r'\bmois d[e\']\b', date_texte.lower())
        if len(splits) > 0:
            _m = splits[-1].strip().split(' ')[0]
            for _mois in mois:
                if ratio(_mois, _m) > 0.5:
                    m = _mois
                    
                    
    if not m and j:
        splits = re.split(j, date_texte.lower())
        if len(splits) > 0:
            _m = splits[-1].strip().split(' ')[0]
            for _mois in mois:
                if ratio(_mois, _m) > 0.5:
                    m = _mois
                    
    if not m:
        words = date_texte.lower().split(' ')
        for word in words:
            if word == 'maire':
                continue
            for _mois in mois:
                if ratio(_mois, word) > 0.7:
                    m = _mois
    return m, j

get_date_registre(actes[i])

('avril', 'huit')

In [114]:
df.loc[:, 'registre_mois'] = df.loc[:, 'acte_pp'].apply(lambda x: get_date_registre(x)[0])
df.loc[:, 'registre_jour'] = df.loc[:, 'acte_pp'].apply(lambda x: get_date_registre(x)[1])
df.loc[:, ['registre_mois', 'registre_jour']].notnull().mean()*100

registre_mois    97.356851
registre_jour    98.264783
dtype: float64

In [115]:
notnulls = df.drop(labels='commune', axis=1).notnull().groupby(df.commune, sort=False).sum()
notnulls.loc[:, 'registre_mois'] =notnulls.loc[:, ['registre_mois', 'acte']].apply(lambda x: x[0]/x[1] * 100,
                                                                                   axis=1)
notnulls.loc[:, ['registre_mois']].sort_values('registre_mois')

,registre_mois
commune,
19. Prêcheur,93.221871
14. Grand Bourg de MG,95.049505
17. Moule,95.079220
3. Basse Pointe,95.202791
14. Lorrain,95.306535
4. Carbet,95.960740
27. Sainte Luce,96.051333
12. Gourbeyre,96.748507
15. Macouba,97.012987


In [116]:
_communes = notnulls.loc[:, ['registre_mois']].sort_values('registre_mois').index[:3]
for x in df.loc[(df.loc[:, 'registre_mois'].isnull()) &
                (df.loc[:, 'commune'].isin(_communes)), 'acte_pp'].sample(10):
    
    x = re.split(r'((?:[Pp]ar-devant|(?:est|sont) comparu))', x)[0]
    print(x)
    print('---')
    regex = r'(?:{})\b'.format('|'.join(mois))
    match = re.search(regex, x.lower())
        
    if match:
        print(match.group())
    print('------------------')
    print('------------------')

L'an Mil huit cent cinquante Six, le dernier 
---
------------------
------------------
Aujourd'hui trois de relevée mil huit cent cinquante-cin à l'heure de midi, 
---
------------------
------------------
L'an mil huit cent cinquante vingt Commune, 
---
------------------
------------------
Aujourd'hui Juge appléant et du arrêté mil huit cent quatre-vingt-cinq, à heures Desbonnes, par
---
------------------
------------------
Aujourd'hui quatorze mil huit cent cinquante-cinq à naîcheures du matin, 
---
------------------
------------------
L'an mil huit cent cinquante, cinq et le vingt-cinq trent, domicilié, 
---
------------------
------------------
Aujourd'hui Népublique Française mil huit cent quatre-vingt-seize à heures du nom par
---
------------------
------------------
Aujourd'hui vingt quatre était mil huit cent quatre vingt treize, à neuf heures du matin, par devant Nous, Virsel Etienne, premier adjoint au Maire, de légué aux fonctions d'officier de l'Etat-Civil de la commun

In [117]:
mois = df.loc[:, 'registre_mois'].values
year = df.loc[:, 'year'].values
cleaned_mois = []
for j, _mois in enumerate(mois):
    if j == 0 or j == len(mois) - 1:
        cleaned_mois.append(_mois)
        continue
    if _mois:
        cleaned_mois.append(_mois)
    else:
        
        l, k = j, j
        while not mois[l]:
            l -= 1
        while not mois[k]:
            k += 1

        if mois[l] == mois[k]:
            cleaned_mois.append(mois[l])
        elif mois[l] == 'décembre' and mois[k] == 'janvier' and (year[l] == year[j] or year[k] == year[j]):
            if year[l] == year[j]:
                cleaned_mois.append(mois[l])
            else:
                cleaned_mois.append(mois[k])
        else:
            cleaned_mois.append(_mois)

df.loc[:, 'registre_mois'] = cleaned_mois
df.loc[:, ['registre_mois', 'registre_jour']].notnull().mean()*100

registre_mois    98.980202
registre_jour    98.264783
dtype: float64

In [118]:
df

,acte,photo,commune,year,acte_pp,registre_mois,registre_jour
0,Premier juillet,FRANOM49_1DPPC0954_225,Vieux Habitants,1855,Premier juillet,juillet,premier
1,"Aujourd'hui, Samedi Six Janvier mil huit cent ...",FRANOM49_1DPPC0954_225,Vieux Habitants,1855,"Aujourd'hui, Samedi Six Janvier mil huit cent ...",janvier,six
2,"Aujourd'hui, Lundi huit Janvier mil huit cent ...",FRANOM49_1DPPC0954_225,Vieux Habitants,1855,"Aujourd'hui, Lundi huit Janvier mil huit cent ...",janvier,un
3,"Aujourd'hui, Lundi huit Janvier mil huit cent ...",FRANOM49_1DPPC0954_226,Vieux Habitants,1855,"Aujourd'hui, Lundi huit Janvier mil huit cent ...",janvier,un
4,"Aujourd'hui, Samedi treize Janvier mil huit ce...",FRANOM49_1DPPC0954_226,Vieux Habitants,1855,"Aujourd'hui, Samedi treize Janvier mil huit ce...",janvier,treize
...,...,...,...,...,...,...,...
76873,"L'an mil huit cent quatre-vingt-quinze, le hui...",FRANOM21_1DPPC1705_078,16. Marigot,1895,"L'an mil huit cent quatre-vingt-quinze, le hui...",décembre,huit
76874,"L'an mil huit cent quatre vingt-quinze, le neu...",FRANOM21_1DPPC1705_078,16. Marigot,1895,"L'an mil huit cent quatre vingt-quinze, le neu...",décembre,huit
76875,"L'an mil huit cent quatre vingt-quinze, le onz...",FRANOM21_1DPPC1705_079,16. Marigot,1895,"L'an mil huit cent quatre vingt-quinze, le onz...",décembre,huit
76876,"L'an mil huit cent quatre vingt quinze, le sei...",FRANOM21_1DPPC1705_079,16. Marigot,1895,"L'an mil huit cent quatre vingt quinze, le sei...",décembre,huit


**Type Évènement**

In [119]:
i = np.random.randint(len(actes))

def get_act_type(acte):
        
    acte = acte.lower()
    
    if re.search('enfant san(?:s|) vie', acte):
        return 'naissance/mort'

    deces = ['d[eé]c[eé]d[ée](?:e|)', 'décès', 'défunt', 'nous en somme(?:s|) assuré',
             '(?:somme(?:s|)|être) assuré', 'nous nous(.*?)assuré']
    naissance = ['te de naissance', 'présenté', 'a(?: été|) présent[eé]', 
                 'nous présente', 'le(?:s|) prénom(?:s|)', "est né d'elle", 'est accouché',
                 'est sort[ie] du sein','accouch','est né',
                "l'enfant a été"
                ]
    mariage = ['mariage', 'c[eé]l[eé]bration', '[eé]poux','unis par les liens', 
               'mari et (pour)* femme','consentant']
    reconnaissance = ['reconna[iî]tre', 'reconnaît','reconnait [pm]ère', 
                      'reconnaissance', 'reconnaissant', 
                       "registres de (?:naissance|l'[eé]tat civil)",
                      'se reconnait être', 'reconnait pour']

    n_deces = len(re.findall(r'(?:{})'.format('|'.join(deces)), acte))
    n_naissance = len(re.findall(r'(?:{})'.format('|'.join(naissance)), acte))
    n_mariage = len(re.findall(r'(?:{})'.format('|'.join(mariage)), acte))

    if n_deces > n_naissance and n_deces > n_mariage:
        return 'décès'
    
    if n_mariage > n_naissance and n_deces < n_mariage:
        return 'mariage'
    
    if n_deces < n_naissance and n_naissance > n_mariage:
        if 'registre de naissance' in acte:
            return 'reconnaissance'
        else:
            return 'naissance'
    
    if n_deces == n_naissance and n_naissance > n_mariage:
        return 'naissance/mort'
    
    if n_deces == n_naissance and n_naissance == n_mariage and n_mariage == 0:
        n_reconnaissance = len(re.findall(r'(?:{})'.format('|'.join(reconnaissance)), acte))
        if n_reconnaissance > 0:
            return 'reconnaissance'
        
    
    return None
acte = df.sample(1).loc[:, 'acte_pp'].values[0]
#acte = "L'an mil huit cent cinquante trois, le douzième jour du mois de Janvier, dix heures du matin, Par-devant nous Amelée Bertrand (Roussel Bonneterre adjoint, remplissant les fonctions de Maire et d'officier de l'état civil de la commune du sieur fort Saint Louis Marie Galante Est comparu le sieur Exideuil Léard agé de trente un ans, cultivateur domicilié en la commune du cin fort Saint Louis, Lequel nous a déclaré que le dix de a mois se onze heures du matin, il est né un enfant du Sexe masculin qu' nous présente et auquel il donne le prénom de Herrot, se reconnai sant pour être le père de cet enfant et l'avoir eu de la demoiselle Eliette Broette agée de trente ans, cultivatrice domiciliée en la susdite commune sur habitation Létang Long. Lequel enfant est né au dit lieu. Les dites déclaration et présentation faites en présence des sieurs Rosmin Latrimail agé de trente sept ans et Saturnin alsalon agé de vingt un ans, tous deux cultivateurs domiciliés en la sus dite commune. Et après lecture faite nous avons seul signé le présent acte de naissance, le père et les témoins ayant déclaré ne savoir écrix ni signer de ce requis Amédée R. Bonneterre"
type_acte = get_act_type(acte)
type_acte

'naissance/mort'

In [120]:
df.loc[:, 'registre_type'] = df.loc[:, 'acte_pp'].apply(lambda x: get_act_type(x))
df.loc[:, ['registre_type', 'registre_mois', 'registre_jour']].notnull().mean()*100

registre_type    99.088166
registre_mois    98.980202
registre_jour    98.264783
dtype: float64

In [121]:
df.registre_type.value_counts()

naissance         35966
décès             29746
mariage            7303
naissance/mort     1969
reconnaissance     1193
Name: registre_type, dtype: int64

In [122]:
notnulls = df.drop(labels='commune', axis=1).notnull().groupby(df.commune, sort=False).sum()
notnulls.loc[:, 'registre_type'] =notnulls.loc[:, ['registre_type', 'acte']].apply(lambda x: x[0]/x[1] * 100,
                                                                                   axis=1)
notnulls.loc[:, ['registre_type']]

,registre_type
commune,
Vieux Habitants,99.332698
8. Capesterre de MG,99.539363
23. Saint Esprit,99.195279
3. Basse Pointe,99.258613
Saint Louis,99.149338
1. Abymes,99.360341
14. Lorrain,99.326297
16. Morne à l'Eau,99.267936
21. Rivière Salée,98.560411


In [123]:
_communes = ['17. Moule', '31. Trois Ilets', '23. Saint Esprit']
for x in df.loc[(df.loc[:, 'registre_type'].isnull()) &
                (df.loc[:, 'commune'].isin(_communes)), 'acte_pp'].sample(10):
    
    print(x)
    deces = ['d[eé]c[eé]d[ée](?:e|)', 'décès', 'défunt', 'nous en somme(?:s|) assuré', 'être assuré']
    naissance = ['acte de naissance', 'présenté', 'a(?: été|) présent[eé]', 
                 'nous présente', 'le(?:s|) prénom(?:s|)', "est né d'elle", 'est accouché',
                 'est sort[ie] du sein','accouch','est né']
    mariage = ['mariage', 'c[eé]l[eé]bration', '[eé]poux','unis par les liens', 
               'mari et (pour)* femme','consentant']
    reconnaissance = ['reconna[iî]tre', 'reconnaît','reconnait [pm]ère', 
                      'reconnaissance', 'reconnaissant']

    n_deces = len(re.findall(r'(?:{})'.format('|'.join(deces)), x))
    n_naissance = len(re.findall(r'(?:{})'.format('|'.join(naissance)), x))
    n_mariage = len(re.findall(r'(?:{})'.format('|'.join(mariage)), x))
        
    print(n_deces, n_naissance, n_mariage)
    print('------------------')
    print('------------------')

L'an mil huit cent cinquante Six et le vingt quatre décembre, à dix heures du matin, par devant nous Joseph Antoine Ifis Desbonne, maire, adjoint de l'état civil de la Commune de Moule, île Guadeloupe est comparu le sieur Louis Pascal, âgé de vingt cinq ans, commis, domicilié en cette Commune, et auquel nous a présent un enfant de sexe masculin, né le quatorze du ce mois, à midi, en cette vissi, Granderne, 4°37, de son légitime Mariageavec la dame Jeannaine présette, âgée de trente deux ans, propriétaire, et auquel il a déclaré nuloir donner les prénoms de Lucien Alexis Victor Camille, lesdites déclaration et présentation faites en présence des sieurs frénoms Inget agé de quarante Six ans et Paul Armand, âgé de vingt deux ans, tous deux Sans profession, demeurant en cette Commune, qui ont signé avec nous et le comparant le présent acte, après lecture Marget Teneau N°44 J. Hte Adrien N°44
0 1 0
------------------
------------------
L'an mil huit cent cinquante cinq et le mardi seize du 

In [124]:
naissances = df.loc[df.loc[:, 'registre_type'] == 'naissance', :].copy()
deces = df.loc[df.loc[:, 'registre_type'] == 'décès', :].copy()
mariages = df.loc[df.loc[:, 'registre_type'] == 'mariage', :].copy()
reconnaissances = df.loc[df.loc[:, 'registre_type'] == 'reconnaissance', :].copy()
mortsnes = df.loc[df.loc[:, 'registre_type'] == 'naissance/mort', :].copy()

In [125]:
## Pour regarder des exemples: changer naissance par deces, mariages, reconnaissance, mortsnes

for x in naissances.sample(10).loc[:, 'acte_pp'].values:
    print(x)
    print('---')

L'an mil huit cent cinquante deux et le samedi premier jour du mois de Mai, à quatre heures après midi. Par-devant nous Jacques adolphe de sagrange chancel, Maire de la commune du Robert canton de la Trinité, arrondissement de Saint Pierre, île Martinique, officier de l'Etat civil de la dite commune. Est comparu les demoiselle Marie Ange Lauhon, âgée de trente quatre ans, cultivatrice, domiciliée en cette Commune laquelle nous a déclaré que le premier du mois d'avril dernier, à quatre heures du matin, elle est accouchée dans sa maison sur l'habitation dite Sainte Savanne, sise en cette commune, d'un enfant du sexe marcelin qu'elle nous présente et auquel elle donne le prénom de Eugène, se reconnaissant pour être le mère de ch enfant. Les dites déclaration et présentation faites en présence des sieurs, adjoint Lauhon, âgé de quarante deux ans, cultivateur et Edouard Vallier, âgé de quarante sept ans, concierge de la mairie, tous deux domiciliés en cette Commune et nous avons seul signé 

In [126]:
naissances.to_csv(join(PATH_REPO, 'data', '20230817_transcriptions', 'naissancesRaw.csv'), index=False)
deces.to_csv(join(PATH_REPO, 'data', '20230817_transcriptions', 'decesRaw.csv'), index=False)
mariages.to_csv(join(PATH_REPO, 'data', '20230817_transcriptions', 'mariagesRaw.csv'), index=False)
reconnaissances.to_csv(join(PATH_REPO, 'data', '20230817_transcriptions', 'reconaissancesRaw.csv'), index=False)
mortsnes.to_csv(join(PATH_REPO, 'data', '20230817_transcriptions', 'mortsNesRaw.csv'), index=False)